# Finite Capacity Region - Constraint Types

This example demonstrates different FCR constraint types:
- Global max jobs: limits total jobs across all classes
- Per-class max jobs: limits jobs of a specific class
- Drop rule: determines blocking vs dropping behavior

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.jmt.*

In [ ]:
// Create a multiclass open network
val model = Network("FCR Constraints Demo")

In [ ]:
// Nodes
val source = Source(model, "Source")
val queue1 = Queue(model, "Queue1", SchedStrategy.FCFS)
val queue2 = Queue(model, "Queue2", SchedStrategy.FCFS)
val sink = Sink(model, "Sink")

In [ ]:
// Job classes
val class1 = OpenClass(model, "HighPriority", 0)
val class2 = OpenClass(model, "LowPriority", 1)

In [ ]:
// Arrival and service rates
source.setArrival(class1, Exp(0.3))  // High priority: lower arrival rate
source.setArrival(class2, Exp(0.5))  // Low priority: higher arrival rate
queue1.setService(class1, Exp(1.0))
queue1.setService(class2, Exp(0.8))
queue2.setService(class1, Exp(1.2))
queue2.setService(class2, Exp(1.0))

In [ ]:
// Routing: both classes go through both queues
val P = model.initRoutingMatrix()
P.set(class1, class1, source, queue1, 1.0)
P.set(class1, class1, queue1, queue2, 1.0)
P.set(class1, class1, queue2, sink, 1.0)
P.set(class2, class2, source, queue1, 1.0)
P.set(class2, class2, queue1, queue2, 1.0)
P.set(class2, class2, queue2, sink, 1.0)
model.link(P)

In [ ]:
// Add Finite Capacity Region with multiple constraints
val fcr = model.addRegion(listOf(queue1, queue2))

// Global constraint: max 2 jobs total in the region
fcr.setGlobalMaxJobs(2)

// Per-class constraints: high priority gets more space
fcr.setClassMaxJobs(class1, 2)  // HighPriority: max 2 jobs
fcr.setClassMaxJobs(class2, 2)  // LowPriority: max 2 jobs
// Note: per-class limits must sum to >= global limit for consistent behavior

// Drop rules: all classes use the same rule (required by LINE)
// true = drop jobs when limit reached, false = block (wait)
fcr.setDropRule(class1, true)   // true = drop
fcr.setDropRule(class2, true)   // true = drop

In [ ]:
// Solve with JMT
val solver = JMT(model, "seed", 23000, "samples", 100000)
val avgTable = solver.avgTable
println("JMT Results:")
avgTable.print()

## Explanation

This example demonstrates the different types of constraints available in Finite Capacity Regions:

1. **Global Max Jobs**: `setGlobalMaxJobs(2)` limits the total number of jobs across all classes in the region to 2.

2. **Per-Class Max Jobs**: `setClassMaxJobs(class, limit)` limits the number of jobs of a specific class in the region.

3. **Drop Rule**: `setDropRule(class, boolean)` determines what happens when the region is full:
   - `true` = Drop jobs (lost)
   - `false` = Block jobs (wait)

The model creates a multiclass open network with two queues inside an FCR, demonstrating how different classes can have different limits while sharing a global capacity constraint.